In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

df = pd.read_csv('annual_dataset.csv')

df.head()

,Year,Population (million),Temperature (celsius),CPI,GDP (billion bahts),Peak Load (MW),Generation (GWh),Consumption (GWh)
0,2002,62.80,26.81,82.3,5769.6,"16,681","111,256","100,091"
1,2003,63.08,26.72,82.4,6184.4,"18,121","117,290","106,987"
2,2004,61.97,26.65,82.8,6573.3,"19,326","121,534","115,101"
3,2005,62.42,26.86,84.1,6848.6,"20,538","134,798","121,240"
4,2006,62.83,26.75,86.0,7188.8,"21,064","141,919","127,879"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   21 non-null     int64  
 1   Population (million)   21 non-null     float64
 2   Temperature (celsius)  21 non-null     float64
 3   CPI                    21 non-null     float64
 4   GDP (billion bahts)    21 non-null     float64
 5   Peak Load (MW)         21 non-null     object 
 6   Generation (GWh)       21 non-null     object 
 7   Consumption (GWh)      21 non-null     object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ KB


In [4]:
df['Peak Load (MW)'] = df['Peak Load (MW)'].str.replace(',', '').astype('float64')
df['Generation (GWh)'] = df['Generation (GWh)'].str.replace(',', '').astype('float64')
df['Consumption (GWh)'] = df['Consumption (GWh)'].str.replace(',', '').astype('float64')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   21 non-null     int64  
 1   Population (million)   21 non-null     float64
 2   Temperature (celsius)  21 non-null     float64
 3   CPI                    21 non-null     float64
 4   GDP (billion bahts)    21 non-null     float64
 5   Peak Load (MW)         21 non-null     float64
 6   Generation (GWh)       21 non-null     float64
 7   Consumption (GWh)      21 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 1.4 KB


In [5]:
df = df.drop(['Year', 'Generation (GWh)', 'Consumption (GWh)'], axis=1)

df = df.sample(frac=1).reset_index(drop=True)

df.head()

,Population (million),Temperature (celsius),CPI,GDP (billion bahts),Peak Load (MW)
0,63.39,26.30,88.90,7710.3,22568.0
1,63.53,26.69,89.10,7657.1,22045.0
2,66.09,26.30,103.04,10680.0,32255.0
3,66.41,26.92,99.48,10693.2,28338.0
4,63.88,27.33,89.99,8232.4,24010.0


In [6]:
X = df.drop('Peak Load (MW)', axis=1).values
y = df['Peak Load (MW)'].values

X.shape, y.shape

((21, 4), (21,))

In [7]:
X_train = X[:-1]
X_test = X[-1:]

y_train = y[:-1]
y_test = y[-1:]

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# import pickle

# pickle.dump(scaler, open('annual_peak_scaler.pkl', 'wb'))

In [10]:
import pickle

scaler = pickle.load(open('annual_peak_scaler.pkl', 'rb'))

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20, 4), (1, 4), (20,), (1,))

In [12]:
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'hidden_layer_sizes': [(16, 8), (32, 16), (64, 32), (16, 16), (32, 32), (64, 64)],
#     'activation': ['tanh', 'relu'],
#     'alpha': [0.001, 0.01, 0.1]
# }

# mlp = MLPRegressor(max_iter=1000)
# grid_search = GridSearchCV(mlp, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# print("Best hyperparameters: ", grid_search.best_params_)
# print("Validation score: {:.3f}".format(grid_search.best_score_))

Best hyperparameters:  {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (64, 64)}
Validation score: -5.709


In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import GridSearchCV

# def create_ANN():
#     model = Sequential()
#     model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(1, activation='linear'))
#     model.compile(loss='mse', optimizer='adam')
#     return model

# model = KerasRegressor(build_fn=create_ANN, verbose=0)

# batch_size = [32, 48, 64]
# epochs = [1000, 3000, 5000]

# param_grid = dict(batch_size=batch_size, epochs=epochs)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid.fit(X_train, y_train)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [13]:
# import tensorflow as tf

# hid_dim = 64
# act_func = 'relu'
# num_epochs = 5000
# batch_size = 32

# model = tf.keras.Sequential([
#         tf.keras.layers.Dense(hid_dim, activation=act_func, input_shape=(X_train.shape[1],)),
#         tf.keras.layers.Dense(hid_dim, activation=act_func),
#         tf.keras.layers.Dense(1)])

# model.compile(loss='mse', optimizer='adam')

# history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/5000
1/1 [==============================] - 1s 763ms/step - loss: 647213568.0000 - val_loss: 712703936.0000
Epoch 2/5000
1/1 [==============================] - 0s 39ms/step - loss: 647210624.0000 - val_loss: 712701056.0000
Epoch 3/5000
1/1 [==============================] - 0s 39ms/step - loss: 647207552.0000 - val_loss: 712698240.0000
Epoch 4/5000
1/1 [==============================] - 0s 40ms/step - loss: 647204608.0000 - val_loss: 712695424.0000
Epoch 5/5000
1/1 [==============================] - 0s 41ms/step - loss: 647201664.0000 - val_loss: 712692736.0000
Epoch 6/5000
1/1 [==============================] - 0s 41ms/step - loss: 647198656.0000 - val_loss: 712690112.0000
Epoch 7/5000
1/1 [==============================] - 0s 38ms/step - loss: 647195648.0000 - val_loss: 712687488.0000
Epoch 8/5000
1/1 [==============================] - 0s 42ms/step - loss: 647192704.0000 - val_loss: 712684800.0000
Epoch 9/5000
1/1 [==============================] - 0s 38ms/step - loss: 647189

In [14]:
# import tensorflow as tf

# tf.keras.models.save_model(model, 'ANN_annual_peak')

INFO:tensorflow:Assets written to: ANN_annual_peak\assets


INFO:tensorflow:Assets written to: ANN_annual_peak\assets


In [15]:
import tensorflow as tf

model = tf.keras.models.load_model('ANN_annual_peak')

In [16]:
test_loss = model.evaluate(X_test, y_test)
print('Test loss:', test_loss)

1/1 [==============================] - 0s 80ms/step - loss: 9537083.0000
Test loss: 9537083.0


In [17]:
y_pred = model.predict(X_test)
print('Predicted value:', y_pred)
print('Actual value:', y_test)

1/1 [==============================] - 0s 83ms/step
Predicted value: [[22414.217]]
Actual value: [19326.]


In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np
import math

def mape(test, pred):
    test, pred = np.array(test), np.array(pred)
    mape = np.mean(np.abs((test - pred) / test))
    return mape

MSE  = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
MAE  = mean_absolute_error(y_test, y_pred)
MAPE = mape(y_test, y_pred)

MSE, RMSE, MAE, MAPE

(9537082.984500885, 3088.216796875, 3088.216796875, 0.15979596382464037)

In [19]:
X_sample = [67, 27, 109, 11000]
X_sample = scaler.transform([X_sample])

y_hat = model.predict([X_sample])
print(y_hat)

1/1 [==============================] - 0s 54ms/step
[[34943.227]]
